# Maxwell (Te) - Supervisor Task Manager Workflow

## 1. Introduction

Maxwell serves as the operational hub of the cognitive assistant, responsible for organizing and managing tasks to ensure they align with overarching existential goals. It utilizes systematic approaches to optimize productivity and drive progress, employing a structured workflow that encompasses input processing, multi-faceted task classification, goal-aligned synthesis, efficient execution, and continuous improvement through feedback analysis.

In [1]:
import dspy
import os
from dotenv import load_dotenv

load_dotenv()

# Load the API key from the .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

gpt_4o_mini = dspy.LM('openai/gpt-4o-mini', temperature=0.7)

dspy.configure(lm=gpt_4o_mini)

/nix/store/faick5k40f2g3210clyk7sfb8wn71s6s-python3.12-pydantic-2.10.3/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


## 2. Input Layer

The Input Layer is the starting point of the workflow, handling the initial reception and processing of tasks or queries.

```mermaid
flowchart LR
    A[Input Received] --> B[Input Processing]
```

### 2.1 Input Received

- The system receives a task or query from the user.

### 2.2 Input Processing

1. Classify: Can you complete the task or answer the question?
    1. If so, return True and stop (answer the question or preform the task)
    2. If not, return False and get more information (ask for clarity or preform web search)
2. Classify: If we need to break this down and continue working
    1. If so continue...
3. Create a plan to answer the question or complete the task.
    1. Return a step by step plan to complete the task or answer the question

In [2]:
# Code for input processing, should check if it's a complete plan.

# Define a module (ChainOfThought) and assign it a signature (return an answer, given a question).
qa = dspy.ChainOfThought('question -> answer')

# Run with the default LM configured with `dspy.configure` above.
response = qa(question="Create a cognitive assistant. Transform the 8 cognitive functions into team members.")
print(response.answer)


The cognitive functions transformed into team members are:

1. **Introverted Intuition (Ni)** - The Visionary
2. **Extraverted Intuition (Ne)** - The Idea Generator
3. **Introverted Sensing (Si)** - The Historian
4. **Extraverted Sensing (Se)** - The Adventurer
5. **Introverted Thinking (Ti)** - The Analyst
6. **Extraverted Thinking (Te)** - The Organizer
7. **Introverted Feeling (Fi)** - The Advocate
8. **Extraverted Feeling (Fe)** - The Harmonizer


In [ ]:
# Check that the plan contains everything we need before getting started
class checkPlanReadiness(dspy.Signature):
    """Verify plan readiness."""

    context: str = dspy.InputField(desc="plans here are assumed to be well thought out and ready to work on")
    text: str = dspy.InputField()
    readiness: bool = dspy.OutputField()
    evidence: dict[str, list[str]] = dspy.OutputField(desc="Supporting evidence for claims")

initial_plan = gpt_4o_mini.history[0]['response'].choices[0].message.content

text = "My next step is to extract all of the tasks for the plan"

readiness = dspy.ChainOfThought(checkPlanReadiness)
readiness(context=initial_plan, text=text)


Prediction(
    reasoning='The plan is well-structured and outlines specific roles that align with the cognitive functions, indicating a clear understanding of how to implement the team dynamics. The next step of extracting tasks is a logical progression that builds on the established roles, suggesting that the plan is ready for execution.',
    readiness=True,
    evidence={'roles': ['Introverted Intuition (Ni) - The Visionary', 'Extraverted Intuition (Ne) - The Idea Generator', 'Introverted Sensing (Si) - The Historian', 'Extraverted Sensing (Se) - The Adventurer', 'Introverted Thinking (Ti) - The Analyst', 'Extraverted Thinking (Te) - The Organizer', 'Introverted Feeling (Fi) - The Advocate', 'Extraverted Feeling (Fe) - The Harmonizer'], 'next_steps': ['Extract tasks for the plan']}
)

In [5]:
print(gpt_4o_mini.history[1]['response'].choices[0].message.content)

[[ ## reasoning ## ]]
The plan to transform the 8 cognitive functions into team roles is well-structured and clearly outlines the characteristics and strengths of each role. Each function has been assigned a corresponding team member role, which demonstrates a comprehensive understanding of the cognitive functions and their application in a team setting. The next step of extracting tasks is a logical progression in the planning process, indicating readiness to move forward.

[[ ## readiness ## ]]
True

[[ ## evidence ## ]]
{"type": "object", "additionalProperties": {"type": "array", "items": {"type": "string"}}, "evidence": {"Roles Defined": ["The Visionary (Ni)", "The Idea Generator (Ne)", "The Historian (Si)", "The Adventurer (Se)", "The Analyst (Ti)", "The Organizer (Te)", "The Advocate (Fi)", "The Harmonizer (Fe)"], "Next Steps": ["Extract tasks for implementation"]}}

[[ ## completed ## ]]


In [10]:
plan_thoughts = gpt_4o_mini.history[0]['response'].choices[0].message.content

# 1) Declare with a signature, and pass some config.
create_plan = dspy.ChainOfThought('plan_thoughts -> detailed_task_list')

# 2) Call with input argument.
response = create_plan(plan_thoughts=plan_thoughts)

# 3) Access the outputs.
response.completions.detailed_task_list

['1. Define the roles based on cognitive functions:\n   - Create detailed descriptions for each role: Visionary, Idea Generator, Historian, Adventurer, Analyst, Organizer, Advocate, Harmonizer.\n   \n2. Develop a team structure:\n   - Assign team members to each role based on their strengths and preferences.\n   - Ensure a balanced representation of all cognitive functions within the team.\n\n3. Establish team objectives:\n   - Identify short-term and long-term goals for the team.\n   - Align team objectives with the strengths of each cognitive function.\n\n4. Create a collaboration plan:\n   - Outline how team members will work together based on their roles.\n   - Schedule regular meetings for brainstorming (led by Ne), strategic planning (led by Ni), and performance reviews (led by Te).\n\n5. Foster team culture:\n   - Encourage the Advocate (Fi) and Harmonizer (Fe) to lead team-building activities.\n   - Promote open communication and feedback among all team members.\n\n6. Monitor a

In [17]:
print(gpt_4o_mini.history[6]['response'].choices[0].message.content)

[[ ## reasoning ## ]]
To effectively transform the 8 cognitive functions into team members, we need to clearly define each role based on their unique characteristics and strengths. This will allow us to establish a balanced team dynamic where each member contributes to the team's success. The defined roles help in understanding how each function operates and interacts with others, ensuring that all perspectives are represented in team discussions and decision-making processes.

[[ ## detailed_task_list ## ]]
1. Define the roles based on cognitive functions:
   - Create detailed descriptions for each role: Visionary, Idea Generator, Historian, Adventurer, Analyst, Organizer, Advocate, Harmonizer.
   
2. Develop a team structure:
   - Assign team members to each role based on their strengths and preferences.
   - Ensure a balanced representation of all cognitive functions within the team.

3. Establish team objectives:
   - Identify short-term and long-term goals for the team.
   - Align

## 3. Classification Layer

The Classification Layer analyzes the processed input from multiple perspectives to gain a comprehensive understanding of the task.

```mermaid
flowchart TD
    C{Task Classification} --> D[Sophia: Strategic Vision]
    C --> E[Isabella: Ethical Considerations]
    C --> F[Evelyn: Logical Analysis]
    C --> G[Serena: Real-time Data]
    C --> H[Diana: Historical Context]
    C --> I[Nova: Creative Ideas]
```

### 3.1 Task Classification

- The processed input is classified to determine its nature, urgency, and requirements.
- Prioritize tasks based on urgency and importance.

### 3.2 Multiple Perspectives Analysis

The task is analyzed from different viewpoints:
- Sophia: Evaluates the strategic implications and long-term vision.
- Isabella: Considers ethical aspects and potential moral implications.
- Evelyn: Performs logical analysis and structured problem-solving.
- Serena: Incorporates real-time data and current context.
- Diana: Provides historical context and past experiences.
- Nova: Offers creative and innovative approaches.


In [ ]:
# Code for Classification

## 4. Synthesis Layer

The Synthesis Layer combines the classified information and ensures alignment with overall goals.

```mermaid
flowchart TD
    J(Task Synthesis) --> K[Goal Alignment Check]
    K -->|Aligned| L(Execution Planning)
    K -->|Not Aligned| M[Reassessment]
    M --> J
```

### 4.1 Task Synthesis

- Information from various perspectives is combined to form a comprehensive understanding of the task.
- Create a structured workflow that outlines task dependencies and sequences.

### 4.2 Goal Alignment Check

- The synthesized task is checked against overall goals and objectives of the system.
- Continuously assess tasks against long-term missions and values.
- Ensure that each task contributes to the overall objectives of the assistant.

### 4.3 Execution Planning

- If aligned, an execution plan is created, outlining steps, resources, and timelines.
- Make recommendations to eliminate problems and work with the user to implement those recommendations.

### 4.4 Reassessment

- If not aligned, the task is reassessed, and the process loops back to synthesis.

In [ ]:
# Code for Synthesis

## 5. Execution Layer

The Execution Layer focuses on selecting and utilizing appropriate tools to carry out the task.

```mermaid
flowchart TD
    N{Tool Selection} --> O[Workflow Automation]
    N --> P[Search Tools]
    N --> Q[Code Interpreter]
```

### 5.1 Tool Selection

- Based on the task requirements, appropriate tools are selected from the available options.

### 5.2 Tool Utilization

The selected tools are used to execute the task:
- Workflow Automation: For streamlining repetitive processes and enhancing efficiency.
- Search Tools: To gather relevant information and resources.
- Code Interpreter: To implement tasks that require coding or programming solutions.
- Calendar Management: To maintain schedule integrity and alert the user to consequences of actions in the context of current workload.
- Interagent Communication: To exchange information and requests between PExA agents.

### 5.3 Execution Monitoring

- Employ execution monitoring and prediction to reason about the course of a task's execution and adapt to unexpected events.

In [ ]:
# Code for Execution

## 6. Output Layer

The Output Layer evaluates the outcome, generates output, and ensures continuous improvement.

```mermaid
flowchart TD
    R(Outcome Evaluation) -->|Successful| S[Output Generation]
    R -->|Needs Improvement| T[Feedback Analysis]
    T --> U[Continuous Improvement]
    U --> B[Input Processing]
    S --> V[Task Completion]
```

### 6.1 Outcome Evaluation

- The result of the task execution is evaluated for success and quality.
- Monitor task completion and evaluate outcomes for future improvements.

### 6.2 Output Generation

- If successful, the final output is generated and presented to the user.

### 6.3 Feedback Analysis

- If improvement is needed, feedback is analyzed to identify areas for enhancement.
- Collect data on task performance and outcomes.

### 6.4 Continuous Improvement

- Insights from feedback are used to improve future processes, feeding back into the Input Processing stage.
- Analyze results to refine strategies and enhance task management processes.
- Incorporate user guidance on task-solving strategies and system autonomy.

### 6.5 Task Completion

- The task is marked as complete, and the system is ready for the next input.

In [ ]:
# Code for Output